## preprocess abdominal MRI data from 3 datasets (KORA, NAKO, UKB) 

### preprocessing steps:
1. combine scans for UKB -- separate .ipynb for that part. (since UKB is split into different scans for the abdomen. differs between scans, but most have 2 scans (upper and lower abdomen) that are relevant for us. combination with simple linear function in the overlap part. so far all scans have to be manually checked after the combination.
2. combine segmentations --> see separate .ipynb for that part. (also handling different shapes of segmentations, different orientations, overlaps etc). Output of this step are image scans (input_scan.nii.gz and combined segmentation map: combined_segmentation.nii.gz, all in RAS orientation)
3. this file: preprocessing 1: loads the datasets KORA, UKB or NAKO (or all 3), and resamples them to a common resolution of 2,2,3mm 
4. preprocessing 2: loads resampled files and normalizes the images by min max normalization. Then saves these images as nii files.

Next step would be creating h5 files to train the quickNat

In [2]:
import os

import nibabel  # for loading and saving nifty files
import torch
from math import ceil, floor
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import Pad
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import nrrd
from nibabel.affines import from_matvec, to_matvec, apply_affine

from skimage import io
import matplotlib.pyplot as plt


#### print(os.getcwd())

### labels:

liver = 1  
spleen = 2  
kidney_r = 3  
kidney_l = 4  
adrenal_r = 5  
adrenal_l = 6  
pancreas = 7  
gallbladder = 8

### load datasets:

directory structure should be like: .../datasets/dataID/data/patID/input_scan.nii.gz and   .../datasets/dataID/data/patID/combined_segmentation.nii.gz  
    
where dataID = KORA, NAKO or UKB  
patIDs for training are in ../datasets/dataID/dataID.train  
e.g. for KORA: .../datasets/KORA/KORA.train  

this has to be run separately for train, test and val split that are indicated in dataID.train, dataID.val, dataID.test files  


In [1]:
root_dir = '/home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets/'
all_images_train = {'KORA': [],
             'NAKO': [],
             'UKB' : []}
all_pat_ids = {'KORA': [],
             'NAKO': [],
             'UKB' : []}
for i, data_id in enumerate(['KORA']):
    data_dir = os.path.join(root_dir, data_id, 'data')
    filenames = os.path.join(root_dir, data_id, data_id+'.test')
    # read filenames
    with open(filenames) as f:
        pat_ids = f.read().splitlines()   
    all_pat_ids[data_id] = pat_ids

    print(pat_ids)
    for folder in pat_ids:
        seg_dir = os.path.join(data_dir,folder)
        #listFiles = os.listdir(seg_dir)
        #seg_files = [f for f in listFiles if "input_scan" in f or 'combined_segmentation' in f]
        try:
            img = nibabel.load(os.path.join(seg_dir,'input_scan.nii.gz'))
            segm = nibabel.load(os.path.join(seg_dir,'combined_segmentation.nii.gz'))
            
            print(segm.get_fdata().dtype)
            segm_test = segm.get_fdata().astype('int')
            print(segm_test.dtype)
        except FileNotFoundError as err:
            print('did not find input scan AND segmentation for patient id: ', folder)
            print(err)
        
        all_images_train[data_id].append((img,segm))
#print(all_images_train)
print(all_pat_ids)

NameError: name 'os' is not defined

### preprocessing 1 (resampling) can be skipped. if already resampled proceed with preprocessing 2

first checks if segmentations have no overlaps  
if overlaps are found than this needs to be handled first  
then resamples the images to 2,2,3mm resolution --> takes a while


In [28]:
for i, data_id in enumerate(['KORA', 'NAKO', 'UKB']):
    print(data_id)
    images = all_images_train[data_id]

    #ax = plt.hist(images.ravel(), bins = 256)
    #plt.show()
    for j, imgs in enumerate(images):
        img_data = imgs[0].get_fdata()
        img_header = imgs[0].header
        segm_data = imgs[1].get_fdata()
        segm_header = imgs[1].header
        print(np.min(segm_data))
        print(np.max(segm_data))
        if np.max(segm_data) > 8:
            print('max bigger than 8 for: ', data_id, all_pat_ids[data_id][j] )
            print ('max: ', np.max(segm_data))          
            print(np.where(segm_data==np.max(segm_data)))
        else:
            print('no overlaps in segmentation')
        
        print('min intensity: ', np.min(img_data))
        print('max intensity: ', np.max(img_data))
        
        steps = img_header['pixdim'][1:4]
        target_voxel_dim = [2,2,3]
        print('steps: ', steps)
        
        segm_data = segm_data.astype('int')
        
        print(np.sum(segm_data == 4))
        if data_id == 'KORA':
            img_data = np.flip(np.moveaxis(img_data, 2, 1))  # 012 -> 021
            segm_data = np.flip(np.moveaxis(segm_data, 2, 1))  # 012 -> 021
            
        img_data, img_header = do_interpolate(img_data, steps, img_header, target_voxel_dim)
        segm_data, segm_header = do_interpolate(segm_data, steps, segm_header, target_voxel_dim, is_label=True)
        print(np.sum(segm_data == 4))
        #print(np.unique(segm_data))
        
        segm_data = segm_data.astype('int')
        print(np.sum(segm_data == 4))

        
        print('shape before: ', img_data.shape)
        # now reshape the input scans to a target dimension dim % 16 = 0 so all images have the same shape
        target_dim = [256,256,128]

        img_data, segm_data = post_interpolate(img_data, labelmap=segm_data, target_shape=target_dim)
        
        print('shape after: ', img_data.shape)
        img_header['dim'][1:4] = img_data.shape
        segm_header['dim'][1:4] = segm_data.shape
        
        



        print('AFFINE: ', img_header.get_best_affine())
        #save images:
        
        #new_img_nii = nibabel.Nifti1Image(img_data, img_header.get_best_affine(), img_header)
        new_img_nii = nibabel.MGHImage(img_data, img_header.get_best_affine(), img_header)
        new_file_path = os.path.join(root_dir,data_id,'data', all_pat_ids[data_id][j],'resampled_image.nii.gz')
        nibabel.save(new_img_nii, new_file_path)
        
        #new_img_nii = nibabel.Nifti1Image(segm_data, segm_header.get_best_affine(), segm_header)
        new_img_nii = nibabel.MGHImage(segm_data, segm_header.get_best_affine(), segm_header)
        new_img_nii.set_data_dtype('uint8')
        new_file_path = os.path.join(root_dir,data_id,'data', all_pat_ids[data_id][j],'resampled_segm.nii.gz')
        nibabel.save(new_img_nii, new_file_path)

KORA
0.0
8.0
no overlaps in segmentation
min intensity:  0.0
max intensity:  863.0
steps:  [1.6666666 1.6666666 1.699997 ]
21388
8418
8418
shape before:  (240, 133, 163)
shape after:  (256, 256, 128)
AFFINE:  [[   2.            0.            0.         -261.92883301]
 [   0.            2.            0.          210.32397461]
 [   0.            0.            3.          -97.33310699]
 [   0.            0.            0.            1.        ]]
NAKO
UKB


### preprocessing 2

scale the pixel values between 0 and 1 (min max normalization)

### load data again (this time load the resampled scans)

In [29]:
root_dir = '/home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets/'
all_images_train = {'KORA': [],
             'NAKO': [],
             'UKB' : []}
all_pat_ids = {'KORA': [],
             'NAKO': [],
             'UKB' : []}
for i, data_id in enumerate(['KORA']):
    data_dir = os.path.join(root_dir, data_id, 'data')
    filenames = os.path.join(root_dir, data_id, data_id+'.test')
    # read filenames
    with open(filenames) as f:
        pat_ids = f.read().splitlines()   
    all_pat_ids[data_id] = pat_ids

    print(pat_ids)
    for folder in pat_ids:
        seg_dir = os.path.join(data_dir,folder)
        #listFiles = os.listdir(seg_dir)
        #seg_files = [f for f in listFiles if "input_scan" in f or 'combined_segmentation' in f]
        try:
            img = nibabel.load(os.path.join(seg_dir,'resampled_image.nii.gz'))
            segm = nibabel.load(os.path.join(seg_dir,'resampled_segm.nii.gz'))
        except FileNotFoundError as err:
            print('did not find input scan AND segmentation for patient id: ', folder)
            print(err)
        
        all_images_train[data_id].append((img,segm))
#print(all_images_train)
print(all_pat_ids)

['2460249']
{'KORA': ['2460249'], 'NAKO': [], 'UKB': []}


### min max normalization for each dataset separately

In [30]:
for i, data_id in enumerate(['KORA', 'NAKO', 'UKB']):
    print(data_id)
    images = all_images_train[data_id]
    
    for j, imgs in enumerate(images):
        img = imgs[0].get_fdata()
        img_header = imgs[0].header
        
        img_min = np.min(img)
        img_max = np.max(img)
        img = (img - img_min)/(img_max -img_min)
                
        new_img_nii = nibabel.Nifti1Image(img, img_header.get_best_affine(), img_header)        
        new_file_path = os.path.join(root_dir,data_id,'data', all_pat_ids[data_id][j],'resampled_normalized_image.nii.gz')
        nibabel.save(new_img_nii, new_file_path)

        


KORA
NAKO
UKB


### some helper functions 

In [5]:
import scipy.interpolate as si

def do_interpolate(source, steps, header, target_voxel_dim, is_label=False):
    x, y, z = [steps[k] * np.arange(source.shape[k]) for k in range(3)]
    #print('x ', x)
    #print('y ', y)
    #print('z ', z)
    
    if is_label:
        method = 'nearest'
    else:
        method = 'linear'

    f = si.RegularGridInterpolator((x, y, z), source, method=method)

    dx, dy, dz = target_voxel_dim
    affine = header.get_best_affine()
    np.fill_diagonal(affine, target_voxel_dim + [1])
    header.set_sform(affine)
    new_grid = np.mgrid[0:x[-1]:dx, 0:y[-1]:dy, 0:z[-1]:dz]
    new_grid = np.moveaxis(new_grid, (0, 1, 2, 3), (3, 0, 1, 2))
    
    
    
    interpolated = f(new_grid)
    
   
    if is_label:
        #print(np.unique(interpolated))
        #print(np.mean(interpolated))
        #print(np.max(interpolated))
        #print('3s ', np.sum(interpolated ==3.98))
        
        #print('kidney l ', interpolated[56,68,34])
        #print(np.where(interpolated==4))
        interpolated = np.round(interpolated)
        #print('rounded')
        #print('kidney l ', interpolated[56,68,34])

        #print('3s ', np.sum(interpolated ==3.98))
        #interpolated = interpolated.astype('int')
        #print('3s ', np.sum(interpolated ==3))

    return interpolated, header

In [6]:
def find_nearest(source_num_arr, target_array=np.arange(0, 500, 16)):
    array = np.asarray(target_array)
    idxs = [(np.abs(array - source_num)).argmin() for source_num in source_num_arr]
    return [array[idx + 1] for idx in idxs]


def post_interpolate(volume, labelmap=None, target_shape=None):
    volume = do_cropping(volume, target_shape)
    if labelmap is not None:
        labelmap = do_cropping(labelmap, target_shape)
    current_shape = volume.shape
    intended_shape_deficit = target_shape - np.asarray(current_shape)

    paddings = [tuple(
        np.array([np.ceil((pad_tuples / 2) - pad_tuples % 2), np.floor((pad_tuples / 2) + pad_tuples % 2)]).astype(
            'int32')) for pad_tuples in intended_shape_deficit]
    paddings = tuple(paddings)

    volume = np.pad(volume, paddings, mode='constant')
    if labelmap is not None:
        labelmap = np.pad(labelmap, paddings, mode='constant')

    return volume, labelmap

In [7]:
import operator

def do_cropping(source_num_arr, bounding):
    start = list(map(lambda a, da: a // 2 - da // 2, source_num_arr.shape, bounding))
    end = list(map(operator.add, start, bounding))
    for i, val in enumerate(zip(start, end)):
        if val[0] < 0:
            start[i] = 0
            end[i] = source_num_arr.shape[i]
    slices = tuple(map(slice, tuple(start), tuple(end)))
    return source_num_arr[slices]